In [2]:
!pip install geopy

     |████████████████████████████████| 111 kB 2.0 MB/s eta 0:00:01


In [1]:
import requests
import urllib.parse as urlparse
from geopy import distance

In [31]:
class Route:
    def __init__(self, headers={"X-NCP-APIGW-API-KEY-ID": "umg9dmbud3",
           "X-NCP-APIGW-API-KEY": "SfOEeqmpLBYapFIBn6HTtS62DcnDq6T7N5v9dMqX"}):
        self.headers = headers
    
    # 각 출발지 -> 위경도 변환
    def addr_to_xy(self):
        # 주소값 입력
        self.d1_name = input("출발지 1을 입력하세요. : ")
        self.d2_name = input("출발지 2를 입력하세요. : ")
        # URL 설정
        self.d1_url = "https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query={}".format(self.d1_name)
        self.d2_url = "https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query={}".format(self.d2_name)
        # 리퀘스트
        self.response1 = requests.get(self.d1_url, headers=self.headers)
        self.response2 = requests.get(self.d2_url, headers=self.headers)
        # JSON 파싱하여 위경도 추출
        self.d1_x = self.response1.json()["addresses"][0]["x"] 
        self.d1_y = self.response1.json()["addresses"][0]["y"]
        self.d2_x = self.response2.json()["addresses"][0]["x"]
        self.d2_y =  self.response2.json()["addresses"][0]["y"]
        return self.d1_x, self.d1_y, self.d2_x, self.d2_y
    
    # 출발지간 위경도 -> 경로값 변환
    def road_path(self):
        self.d1_xy = str(self.d1_x) + "," + str(self.d1_y)
        self.d2_xy = str(self.d2_x) + "," + str(self.d2_y)
        self.d_option = "traoptimal"
        self.d_url = "https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving?start={}&goal={}&option={}".format(self.d1_xy, self.d2_xy, self.d_option)
        self.response = requests.get(self.d_url, headers=self.headers)
        self.path = self.response.json()["route"]["traoptimal"][0]["path"]
        self.avg_path = self.response.json()["route"]["traoptimal"][0]["summary"]["distance"] / len(self.path)
        print("거리 : {}m, ".format(self.response.json()["route"]["traoptimal"][0]["summary"]["distance"]), len(self.path), ", ", self.avg_path)
        return self.path
    
    # 출발지간 대중교통 경로
    def trans_path(self, path_type=0):
        self.path_type = path_type
        self.key = 'lxOFkKZ6BCIrJYAQQbeuYsBW+6br+fKss6pEigRpVqA'
        self.url = 'https://api.odsay.com/v1/api/searchPubTransPathT'
        self.params = {'apiKey' : self.key,
                  'SX' : self.d1_x,
                  'SY' : self.d1_y,
                  'EX' : self.d2_x,
                  'EY' : self.d2_y,
                  'SearchPathType' : self.path_type,
                  'OPT':0,
                 }
        self.url = self.url + '?' + urlparse.urlencode(self.params)
        self.response = requests.get(self.url)
        # 경로내 정류장 위경도 추출
        self.transit_count = self.response.json()['result']['path'][0]['info']['busTransitCount'] + self.response.json()['result']['path'][0]['info']['subwayTransitCount']
        self.distance = round(self.response.json()['result']['path'][0]['info']['totalDistance'] / 1000,2)
        self.stop_lat_lng = []
        for i in range(1, 2*self.transit_count, 2):
            self.stop_info = self.response.json()['result']['path'][0]['subPath'][i]['passStopList']['stations']
            self.stop_lat_lng += [(float(a['y']),float(a['x'])) for a in self.stop_info]
        # 경로가 1km이하거나, 정류장이 5개 미만이면 모든 위경도 표출하고, 이외에는 중간 6개의 정류장 위경도만 표출
        if self.distance <= 1 or len(self.stop_lat_lng) <= 5:
            self.meet_point = self.stop_lat_lng
        else:
            self.meet_point = self.stop_lat_lng[len(self.stop_lat_lng)//2-3:len(self.stop_lat_lng)//2+3]
        print('거리: {}km'.format(self.distance))
        return self.meet_point, self.stop_lat_lng

In [32]:
rte = Route()

In [12]:
출발지 1을 입력하세요. : 서울시 강서구 강서로 426
출발지 2를 입력하세요. : 서울시 도봉구 방학2동 344-5


SyntaxError: invalid syntax (<ipython-input-12-041a2a9dc4ca>, line 1)

In [33]:
d1_x, d1_y, d2_x, d2_y = rte.addr_to_xy()

출발지 1을 입력하세요. : 서울시 강서구 강서로 426
출발지 2를 입력하세요. : 서울시 도봉구 방학2동 344-5


In [50]:
d1_x, d1_y, d2_x, d2_y

('126.8406143', '37.5633724', '127.0303294', '37.6672368')

In [51]:
# 대중교통 이동
meet_points, all_points = rte.trans_path()

거리: 37.99km


In [52]:
# 경로내 위, 경도 최대 최소값 구하기
lat_list = [point[0] for point in all_points]
lng_list = [point[1] for point in all_points]

In [53]:
min(lat_list), max(lat_list), min(lng_list), max(lng_list)

(37.542432, 37.667715, 126.827282, 127.038225)

In [54]:
df = pd.read_csv('df.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10922 entries, 0 to 10921
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   10922 non-null  int64  
 1   fname        10416 non-null  object 
 2   name         10416 non-null  object 
 3   tel          10349 non-null  object 
 4   addr         10416 non-null  object 
 5   si           10416 non-null  object 
 6   gu           10416 non-null  object 
 7   dong         10416 non-null  object 
 8   bizhour      10121 non-null  object 
 9   category     10415 non-null  object 
 10  menu         10333 non-null  object 
 11  context      10416 non-null  object 
 12  microreview  10416 non-null  object 
 13  img          10255 non-null  object 
 14  lng          10416 non-null  float64
 15  lat          10416 non-null  float64
 16  n_rating     10922 non-null  float64
 17  b_rating     10922 non-null  float64
 18  mg_rating    10922 non-null  float64
 19  d_ra

In [55]:
df['latlng'] = df.apply(lambda x:(x['lat'], x['lng']), axis=1)
df.tail()

,Unnamed: 0,fname,name,tel,addr,si,gu,dong,bizhour,category,...,img,lng,lat,n_rating,b_rating,mg_rating,d_rating,mn_rating,rating,latlng
10917,11020,반피셋,반피셋,010-5448-9476,서울특별시 강동구 길동 366-1,서울특별시,강동구,길동,"매일 11:30~21:30 Break Time 15:00~17:00 매주 일요일,월...",동남아식,...,https://ldb-phinf.pstatic.net/20191109_241/157...,127.141008,37.539458,4.61,3.53,3.68,1.10,3.54,3.96,"(37.53945820000001, 127.1410081)"
10918,11021,도니도리곱창 길동점,도니도리곱창,02-484-9991,서울특별시 강동구 길동 390-23,서울특별시,강동구,길동,매일 17:00~02:00 | 일요일 17:00~12:00 | 월요일 휴무,한식,...,https://ldb-phinf.pstatic.net/20190401_231/155...,127.140432,37.537137,4.46,3.53,3.68,1.94,3.54,3.95,"(37.5371366, 127.1404324)"
10919,11022,동남집,동남집,02-477-0590,서울특별시 강동구 길동 458-3,서울특별시,강동구,길동,매일 00:00~24:00 월요일 오전 10시 오픈/일요일 오후10시 마감,한식,...,https://ldb-phinf.pstatic.net/20191225_218/157...,127.138280,37.535022,4.27,3.53,3.68,1.94,3.54,3.85,"(37.5350217, 127.1382802)"
10920,11023,라무진 길동점,라무진,02-6397-7299,서울특별시 강동구 길동 414-10,서울특별시,강동구,길동,"매일 17:00~24:00 일요일 정기휴무, 라스트오더 마감1시간전",중식,...,http://ldb.phinf.naver.net/20190412_142/155506...,127.137484,37.536099,4.40,3.53,3.68,1.50,3.54,3.89,"(37.5360989, 127.137484)"
10921,11024,한신포차 길동점,한신포차,02-487-3329,서울특별시 강동구 길동 415-18,서울특별시,강동구,길동,"평일 16:00~06:00 | 주말 16:00~07:00 (금,토)",바,...,https://ldb-phinf.pstatic.net/20180508_137/152...,127.136927,37.536065,4.24,3.53,3.68,1.94,3.54,3.84,"(37.5360649, 127.136927)"


In [56]:
# 경로 square 내 맛집 추출
direction_square = df[(df['lat']<=max(lat_list)) & (df['lat']>=min(lat_list)) & (df['lng']>=min(lng_list)) & (df['lng']<=max(lng_list))]

In [57]:
# square 내 경로 1km 이내 맛집 추출
matzip=pd.DataFrame()
for lat_lng in direction_square['latlng']:
    for point in meet_points:
        dis = distance.distance(point, lat_lng).km
        if dis <= 1:
            df_1 = direction_square[direction_square['latlng'] == lat_lng]
            matzip = matzip.append(df_1).reset_index(drop=True)

In [58]:
matzip

,Unnamed: 0,fname,name,tel,addr,si,gu,dong,bizhour,category,...,img,lng,lat,n_rating,b_rating,mg_rating,d_rating,mn_rating,rating,latlng
0,402,원조순희네빈대떡,원조순희네빈대떡,02-2264-5057,서울특별시 종로구 종로5가 138-9,서울특별시,종로구,종로5가,"평일 08:00~23:00 설날, 추석당일 휴무 | 일요일 08:00~22:00 저...",한식,...,https://ldb-phinf.pstatic.net/20160108_16/1452...,127.000729,37.570594,4.33,3.53,3.68,1.94,3.54,3.88,"(37.5705942, 127.0007294)"
1,403,어머니국시방,어머니국시방,02-2272-9414,서울특별시 종로구 종로5가 407,서울특별시,종로구,종로5가,매일 10:00~16:30,한식,...,https://ldb-phinf.pstatic.net/20150831_21/1440...,127.006543,37.570161,4.50,3.53,0.00,1.94,3.54,3.05,"(37.5701611, 127.0065429)"
2,404,진옥화할매원조닭한마리,진옥화할매원조닭한마리,02-2275-9666,서울특별시 종로구 종로5가 265-22,서울특별시,종로구,종로5가,"매일 10:30~01:00 마지막주문 23:30(휴무:명절 전날,당일)",한식,...,https://ldb-phinf.pstatic.net/20150831_186/144...,127.005739,37.570350,4.31,3.53,3.60,1.94,3.54,3.85,"(37.57035, 127.0057389)"
3,405,제주바다 서울본점,제주바다,010-2499-1193,서울특별시 종로구 효제동 201 1층,서울특별시,종로구,효제동,매일 11:00~24:00 무휴,한식,...,https://ldb-phinf.pstatic.net//20161006_294/14...,127.003981,37.571986,4.30,3.53,3.68,1.94,3.54,3.87,"(37.57198629999999, 127.0039815)"
4,406,박가네 빈대떡,박가네,02-2264-0847,서울특별시 종로구 종로5가 138-10,서울특별시,종로구,종로5가,매일 08:00~22:00 연중무휴,한식,...,https://ldb-phinf.pstatic.net/20170612_61/1497...,127.000825,37.570535,4.31,3.53,3.68,1.94,3.54,3.87,"(37.570535, 127.0008248)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
932,4777,아웃백스테이크하우스 미아점,아웃백스테이크하우스,02-944-2661,서울특별시 강북구 미아동 70-6,서울특별시,강북구,미아동,매일 10:30~21:00 Last Order 20:30 / 11월24일 ~12월7일,양식,...,https://ldb-phinf.pstatic.net/20200108_22/1578...,127.030532,37.614601,4.32,3.53,3.68,1.94,3.54,3.88,"(37.6146006, 127.0305324)"
933,4779,노브랜드버거 미아사거리역점,노브랜드버거,02-945-5280,서울특별시 강북구 미아동 35-30,서울특별시,강북구,미아동,매일 11:00~01:00,양식,...,https://ldb-phinf.pstatic.net/20200327_96/1585...,127.030728,37.611850,4.36,3.53,3.68,1.05,3.54,3.83,"(37.6118504, 127.0307281)"
934,4780,천태막국수,천태막국수,02-985-5255,서울특별시 강북구 미아동 72-18,서울특별시,강북구,미아동,매일 11:00~21:30,한식,...,https://ldb-phinf.pstatic.net/20200822_162/159...,127.031059,37.614653,4.76,3.53,3.68,1.94,3.54,4.10,"(37.6146531, 127.03105890000002)"
935,4783,오센,오센,02-988-0548,서울특별시 강북구 미아동 35-24,서울특별시,강북구,미아동,목요일 17:00~03:00 | 수요일 17:00~03:00 | 화요일 17:00~...,바,...,https://ldb-phinf.pstatic.net/20200529_95/1590...,127.030518,37.612259,4.52,3.53,3.68,0.80,3.54,3.89,"(37.6122593, 127.030518)"
